In [ ]:
import numpy as np
import os 
import sys
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import importlib

In [ ]:
util_path = 'C:/ASM/Dropbox/Developments/Jupyter/Eating/myutils' if 'C:' in os.getcwd() else './myutils'
sys.path.append(util_path)
import my_file_utils as mfileu
import my_classification_utils as mcu
#importlib.reload(mdu)

In [ ]:
train_test = 'train'
if 'C:' not in mfileu.get_path():    
    train_test = sys.argv[1]
    
assert train_test in ['train', 'test_free', 'test_lab']

In [ ]:
#process lab_features
fs_lab = mfileu.read_file('features/features_all_samples', 'lab_features_steven_right.pkl')
ssil_lab = mfileu.read_file('peak_ssil', 'lab_peak_ssil_steven_right.pkl')
ssil_count = len(ssil_lab)
features_lab = np.zeros((ssil_count, 32))
for i in range(ssil_count):
    subj, sess, ix = ssil_lab[i, 0], ssil_lab[i, 1], ssil_lab[i, 2]-40
    features_lab[i, :] = fs_lab[subj][sess][ix, 1:]    

In [ ]:
#process lab_features
fs_free = mfileu.read_file('features/features_all_samples', 'free_features_steven_right.pkl')
ssil_free = mfileu.read_file('peak_ssil', 'free_peak_ssil_steven_right.pkl')
ssil_count = len(ssil_free)
features_free = np.zeros((ssil_count, 32))
for i in range(ssil_count):
    subj, sess, ix = ssil_free[i, 0], ssil_free[i, 1], ssil_free[i, 2]-40
    features_free[i, :] = fs_free[subj][sess][ix, 1:]    

In [ ]:
if train_test=='train':
    for subj in range(11):    
        print("Train Subject: ", subj)

        ### lab
        exclude_subj = -1
        if subj<5:
            exclude_subj = subj+2        
        elif subj>=200:
            exclude_subj = subj-200        

        cond = (ssil_lab[:, 0]!=exclude_subj) & (ssil_lab[:, -1]==1)
        train_x_lab = features_lab[cond, :]
        train_y_lab = ssil_lab[cond, -1]
        print("lab data size: ", ssil_lab.shape, train_x_lab.shape, train_y_lab.shape)

        ### Free
        cond = (ssil_free[:, 0]!=subj) & (ssil_free[:, -1]==0)
        train_x_free = features_free[cond, :]
        train_y_free = ssil_free[cond, -1]
        print("Free data size: ", ssil_free.shape, train_x_free.shape, train_y_free.shape)


        train_x, train_y = np.concatenate((train_x_lab, train_x_free)), np.concatenate((train_y_lab, train_y_free))
        print("Subj:", subj, ", shapes:", train_x.shape, train_y.shape, "Neg, Bite, sip:", np.sum(train_y==0), np.sum(train_y==1))

        clf = RandomForestClassifier(n_estimators =100, random_state=0, n_jobs=-1, verbose=1)    
        clf.fit(train_x, train_y)    
        print("Training done!")

        mfileu.write_file('bite_models_rf', 'subj_'+str(subj)+'.pkl', clf)
        print("Model saved")


In [ ]:
if train_test=='test_free':       
    print("Testing Free ....., all ssil shape: ", ssil_free.shape)    
    ba = mfileu.read_file('data', 'free_data_steven_blank_array.pkl')
    
    for subj in range(11):
        cond = (ssil_free[:, 0]==subj)
        ssil = ssil_free[cond, :]
        test_x_free = features_free[cond, :]        
        assert np.sum(ssil[:, 0]!=subj) == 0

        l = ssil[:, -1]
        print("\n\nSubj, Label summary:", subj, ssil.shape, np.sum(l==0), np.sum(l==1), np.sum(l==2), np.sum(l==3))

        clf = mfileu.read_file('bite_models_rf', 'subj_'+str(subj)+'.pkl')        
        proba = clf.predict_proba(test_x_free)
        proba = np.array(proba)
        proba = proba[:, 1].reshape((-1, 1))
        ssilp = np.concatenate((ssil, proba), axis=1)
        
        for sess in range(len(fs_free[subj])):
            cond = (ssilp[:, 1]==sess)
            ba[subj][sess] = ssilp[cond, :]
            assert np.sum(ba[subj][sess][:, 0]!=subj) == 0
            assert np.sum(ba[subj][sess][:, 1]!=sess) == 0
            
            ix = ssilp[cond, 2]
            assert np.sum((ix[1:]-ix[:-1])<=0)==0
            
            print("Subj, sess, ssilp shape: ", subj, sess, ba[subj][sess].shape)            

    mfileu.write_file('classification_results', 'free_ssilp_rf.pkl', ba)
    
    